Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from kneed import KneeLocator
from numpy.linalg import norm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import sqlite3 as sql

Read  in Data

In [2]:
df = pd.read_csv('telco.csv')

Drop instances with missing values either in MSISDN/Number or Bearer Id 

In [3]:
df = df[df['MSISDN/Number'].notna()]

In [4]:
df = df[df['Bearer Id'].notna()]

In [5]:
df = df.reset_index()

Sort Columns to numerical and categorical

In [6]:
num_cols = []
categorical_cols = []

In [7]:
def sort_cols(data):


    for i in data.columns:
        if data[i].dtypes == "object":
            categorical_cols.append(i)
        else :
            num_cols.append(i)

In [8]:
sort_cols(df)

In [9]:
def fix_cat_cols(data,cat_columns):
    for i in cat_columns:
        data[i] = data[i].fillna(data[i].mode()[0])

In [10]:
fix_cat_cols(df,categorical_cols)

In [11]:
def fix_num_cols(data,num_columns):
    for i in num_columns:
        data[i] = data[i].fillna(data[i].mean())

In [12]:
fix_num_cols(df,num_cols)

Scale numerical columns using standard scaler.

In [13]:
df[num_cols] = StandardScaler().fit_transform(df[num_cols])

# Task 4

Generate engagement features

In [14]:
df['Total_volume (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

In [15]:
df['Session frequency'] = df.groupby(['Bearer Id'])['Dur. (ms)'].transform('count')

In [16]:
df['AVG RTT (ms)'] = df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']
df['Avg Bearer TP (kbps)'] = df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']
df['TCP Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']

In [17]:
needed_cols = ['Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)',"Dur. (ms)",'Total_volume (Bytes)']

Fix outliers in numerical columns

In [18]:
for _ in needed_cols:
    Q1 = df[_].quantile(0.05)
    Q3 = df[_].quantile(0.95)

    df[_] = np.where(df[_] < Q1, df[_].mean(), df[_] )
    df[_] = np.where(df[_] > Q3, df[_].mean(), df[_] )
    print(df[_].skew())

0.744336465155893
0.8871263872840702
1.5683406246562435
2.8292960770832267
-0.8799218231536895
-1.0479368331422163
0.6267722550599922
-0.018707229171748746


Store engagement metrics in column X

In [19]:
X = df[['Session frequency','Dur. (ms)','Total_volume (Bytes)']]

In [20]:
df.describe()

,index,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),...,Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Total_volume (Bytes),Session frequency,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes)
count,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,148506.000000,1.485060e+05,1.485060e+05,1.485060e+05,148506.000000,148506.000000,...,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,148506.000000,1.485060e+05,1.485060e+05,1.485060e+05
mean,6.124296e-17,-4.561644e-16,7.657762e-17,-4.475999e-17,-0.090620,-1.247852e-12,-1.525093e-19,3.487021e-16,-0.067472,-0.080535,...,-2.062165e-17,4.265476e-17,7.827616e-17,-2.451632e-16,1.090890e-16,7.218246e-03,1.240327,2.234411e-17,-5.081252e-17,-6.741510e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,0.561425,1.000003e+00,1.000003e+00,1.000003e+00,0.070952,0.119756,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.090028e+00,0.638478,1.419633e+00,1.765438e+00,1.420845e+00
min,-1.736517e+00,-1.116121e+00,-1.729974e+00,-1.730974e+00,-1.047212,-2.725432e+02,-3.388678e-03,-2.142496e+00,-0.157090,-0.217451,...,-1.733506e+00,-1.731962e+00,-1.732628e+00,-3.391657e+00,-1.832882e+00,-2.323021e+00,1.000000,-4.310445e-01,-9.361938e-01,-2.279397e-01
25%,-8.630556e-01,-9.666889e-01,-8.636097e-01,-8.596554e-01,-0.380642,-1.303239e-02,-3.368152e-03,-5.813090e-01,-0.130656,-0.178220,...,-8.697119e-01,-8.671063e-01,-8.636376e-01,-7.005508e-01,-8.663250e-01,-7.785400e-01,1.000000,-3.090154e-01,-9.242018e-01,-1.793037e-01
50%,-2.094556e-03,-9.666889e-01,-7.109151e-04,4.720407e-03,-0.230461,-3.515589e-03,-3.363094e-03,-5.696471e-01,-0.089118,-0.112833,...,4.286774e-04,2.916946e-03,6.984898e-04,2.002651e-03,5.228838e-03,-1.374139e-16,1.000000,-2.041186e-01,-9.200605e-01,-6.359696e-17
75%,8.673002e-01,1.000835e+00,8.656534e-01,8.690962e-01,0.143047,1.126367e-02,-3.355012e-03,1.677018e+00,0.000000,0.000000,...,8.663118e-01,8.655972e-01,8.634914e-01,7.016198e-01,8.644919e-01,7.823430e-01,1.000000,0.000000e+00,3.015852e-01,-6.359696e-17
max,1.729370e+00,1.050646e+00,1.732018e+00,1.736943e+00,1.688674,3.317321e+00,3.600028e+02,2.251262e+00,0.214866,0.344870,...,1.729251e+00,1.736653e+00,1.739672e+00,3.299398e+00,1.835977e+00,2.322077e+00,16.000000,1.825823e+02,2.100730e+01,1.832445e+02


Use of Kmeans to seperate Engagement metrics to 3 clusters

In [21]:
kmeans = KMeans(
    init= "random",
    n_clusters = 3,
    n_init= 10,
    max_iter= 300,
    random_state= 42
)

In [22]:
kmeans.fit(X)

KMeans(init='random', n_clusters=3, random_state=42)

In [23]:
X['Engagement_labels'] = kmeans.labels_

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/3504866275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Engagement_labels'] = kmeans.labels_


In [24]:
X

,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels
0,1,0.055330,-0.984474,2
1,1,0.055330,1.938198,0
2,1,0.055330,-1.889650,2
3,1,0.055330,1.798275,0
4,1,0.055330,0.240365,1
...,...,...,...,...
148501,1,-0.561190,0.140811,1
148502,1,-0.294231,1.953256,0
148503,1,-0.087749,0.692219,1
148504,2,-0.084308,0.015305,1


In [25]:
X.describe()

,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels
count,148506.000000,148506.000000,1.485060e+05,148506.000000
mean,1.240327,-0.090620,7.218246e-03,1.011090
std,0.638478,0.561425,1.090028e+00,0.739114
min,1.000000,-1.047212,-2.323021e+00,0.000000
25%,1.000000,-0.380642,-7.785400e-01,0.000000
50%,1.000000,-0.230461,-1.374139e-16,1.000000
75%,1.000000,0.143047,7.823430e-01,2.000000
max,16.000000,1.688674,2.322077e+00,2.000000


In [26]:
X1 = X[X.Engagement_labels==1]
X0 = X[X.Engagement_labels==0]
X2 = X[X.Engagement_labels==2]

Fetch columns for experience metrics

In [27]:
needed_cols = ['Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)','TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)']

In [28]:
df.describe()

,index,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),...,Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Total_volume (Bytes),Session frequency,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes)
count,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,148506.000000,1.485060e+05,1.485060e+05,1.485060e+05,148506.000000,148506.000000,...,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,1.485060e+05,148506.000000,1.485060e+05,1.485060e+05,1.485060e+05
mean,6.124296e-17,-4.561644e-16,7.657762e-17,-4.475999e-17,-0.090620,-1.247852e-12,-1.525093e-19,3.487021e-16,-0.067472,-0.080535,...,-2.062165e-17,4.265476e-17,7.827616e-17,-2.451632e-16,1.090890e-16,7.218246e-03,1.240327,2.234411e-17,-5.081252e-17,-6.741510e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,0.561425,1.000003e+00,1.000003e+00,1.000003e+00,0.070952,0.119756,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.090028e+00,0.638478,1.419633e+00,1.765438e+00,1.420845e+00
min,-1.736517e+00,-1.116121e+00,-1.729974e+00,-1.730974e+00,-1.047212,-2.725432e+02,-3.388678e-03,-2.142496e+00,-0.157090,-0.217451,...,-1.733506e+00,-1.731962e+00,-1.732628e+00,-3.391657e+00,-1.832882e+00,-2.323021e+00,1.000000,-4.310445e-01,-9.361938e-01,-2.279397e-01
25%,-8.630556e-01,-9.666889e-01,-8.636097e-01,-8.596554e-01,-0.380642,-1.303239e-02,-3.368152e-03,-5.813090e-01,-0.130656,-0.178220,...,-8.697119e-01,-8.671063e-01,-8.636376e-01,-7.005508e-01,-8.663250e-01,-7.785400e-01,1.000000,-3.090154e-01,-9.242018e-01,-1.793037e-01
50%,-2.094556e-03,-9.666889e-01,-7.109151e-04,4.720407e-03,-0.230461,-3.515589e-03,-3.363094e-03,-5.696471e-01,-0.089118,-0.112833,...,4.286774e-04,2.916946e-03,6.984898e-04,2.002651e-03,5.228838e-03,-1.374139e-16,1.000000,-2.041186e-01,-9.200605e-01,-6.359696e-17
75%,8.673002e-01,1.000835e+00,8.656534e-01,8.690962e-01,0.143047,1.126367e-02,-3.355012e-03,1.677018e+00,0.000000,0.000000,...,8.663118e-01,8.655972e-01,8.634914e-01,7.016198e-01,8.644919e-01,7.823430e-01,1.000000,0.000000e+00,3.015852e-01,-6.359696e-17
max,1.729370e+00,1.050646e+00,1.732018e+00,1.736943e+00,1.688674,3.317321e+00,3.600028e+02,2.251262e+00,0.214866,0.344870,...,1.729251e+00,1.736653e+00,1.739672e+00,3.299398e+00,1.835977e+00,2.322077e+00,16.000000,1.825823e+02,2.100730e+01,1.832445e+02


Store Experience Metrics in Y dataframe

In [29]:
Y = df[['AVG RTT (ms)','Avg Bearer TP (kbps)','TCP Retrans. Vol (Bytes)']]

In [30]:
Y

,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes)
0,-0.286358,-0.925733,-6.359696e-17
1,-0.242932,-0.929912,-6.359696e-17
2,0.000000,-0.934000,-6.359696e-17
3,0.000000,-0.924857,-6.359696e-17
4,0.000000,-0.934000,-6.359696e-17
...,...,...,...
148501,0.277626,2.007879,-7.639381e-02
148502,-0.370625,-0.919989,-6.359696e-17
148503,-0.353911,-0.923573,-6.359696e-17
148504,-0.271393,-0.924251,-6.359696e-17


Use of Kmeans to seperate Experience metrics to 3 clusters

In [31]:
kmeans.fit(Y)

KMeans(init='random', n_clusters=3, random_state=42)

In [32]:
Y['Experience_labels'] = kmeans.labels_

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/1125441398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Experience_labels'] = kmeans.labels_


In [33]:
Y['Experience_labels'].value_counts()

0    127683
2     20112
1       711
Name: Experience_labels, dtype: int64

In [34]:
Y1 = Y[Y.Experience_labels==1]
Y0 = Y[Y.Experience_labels==0]
Y2 = Y[Y.Experience_labels==2]

## Task 4.1

Get the euclidean distance between the engagement metrics and the least engagement label to get Engagement score

In [35]:
X['Engagement_score'] =np.sqrt(((X2['Session frequency'] + X['Dur. (ms)'] + X['Total_volume (Bytes)']).pow(2)) - ((2**2)))
X['Engagement_score'] = X['Engagement_score'].fillna(0)                            

C:\Users\CLASSISH\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/2325902614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Engagement_score'] =np.sqrt(((X2['Session frequency'] + X['Dur. (ms)'] + X['Total_volume (Bytes)']).pow(2)) - ((2**2)))
C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/2325902614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [36]:
X['Engagement_score'].value_counts()

0.000000    147335
0.580058         1
0.299444         1
0.652856         1
0.724395         1
             ...  
4.599944         1
4.772829         1
4.030307         1
0.907907         1
1.669558         1
Name: Engagement_score, Length: 1172, dtype: int64

In [37]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148506 entries, 0 to 148505
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Session frequency     148506 non-null  int64  
 1   Dur. (ms)             148506 non-null  float64
 2   Total_volume (Bytes)  148506 non-null  float64
 3   Engagement_labels     148506 non-null  int32  
 4   Engagement_score      148506 non-null  float64
dtypes: float64(3), int32(1), int64(1)
memory usage: 5.1 MB


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148506 entries, 0 to 148505
Data columns (total 61 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   index                                     148506 non-null  float64
 1   Bearer Id                                 148506 non-null  float64
 2   Start                                     148506 non-null  object 
 3   Start ms                                  148506 non-null  float64
 4   End                                       148506 non-null  object 
 5   End ms                                    148506 non-null  float64
 6   Dur. (ms)                                 148506 non-null  float64
 7   IMSI                                      148506 non-null  float64
 8   MSISDN/Number                             148506 non-null  float64
 9   IMEI                                      148506 non-null  float64
 10  Last Location Name  

In [39]:
Y['Experience_labels'].value_counts()

0    127683
2     20112
1       711
Name: Experience_labels, dtype: int64

Get the euclidean distance between the Experience metrics and the least engagement label to get Experience score

In [40]:
Y['Experience_score'] =np.sqrt(((Y2['AVG RTT (ms)'] + Y2['Avg Bearer TP (kbps)'] + Y2['TCP Retrans. Vol (Bytes)']).pow(2)) - ((2**2)))
Y['Experience_score'] = Y['Experience_score'].fillna(0)

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/158759607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Experience_score'] =np.sqrt(((Y2['AVG RTT (ms)'] + Y2['Avg Bearer TP (kbps)'] + Y2['TCP Retrans. Vol (Bytes)']).pow(2)) - ((2**2)))
C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/158759607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['Experience_score'] = Y['Experience_score'].fillna(0)


In [41]:
Y

,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score
0,-0.286358,-0.925733,-6.359696e-17,0,0.000000
1,-0.242932,-0.929912,-6.359696e-17,0,0.000000
2,0.000000,-0.934000,-6.359696e-17,0,0.000000
3,0.000000,-0.924857,-6.359696e-17,0,0.000000
4,0.000000,-0.934000,-6.359696e-17,0,0.000000
...,...,...,...,...,...
148501,0.277626,2.007879,-7.639381e-02,2,0.938173
148502,-0.370625,-0.919989,-6.359696e-17,0,0.000000
148503,-0.353911,-0.923573,-6.359696e-17,0,0.000000
148504,-0.271393,-0.924251,-6.359696e-17,0,0.000000


In [42]:
Y['Experience_score'].value_counts()

0.000000    133446
2.883482         1
7.407336         1
2.126804         1
2.451135         1
             ...  
1.284306         1
3.056484         1
0.352108         1
3.582946         1
0.938173         1
Name: Experience_score, Length: 15061, dtype: int64

In [43]:
new_df= pd.DataFrame()

In [44]:
new_df['MSISDN/Number'] = df['MSISDN/Number']

In [45]:
new_df = new_df.join(X)
new_df = new_df.join(Y)

In [46]:
new_df['Experience_score'].value_counts()

0.000000    133446
2.883482         1
7.407336         1
2.126804         1
2.451135         1
             ...  
1.284306         1
3.056484         1
0.352108         1
3.582946         1
0.938173         1
Name: Experience_score, Length: 15061, dtype: int64

## Task 4.2

Get the Satisfaction score from the average of engagement and experience score.

In [47]:
new_df['Satisfaction_score'] = ((new_df['Engagement_score'] + new_df['Experience_score'])/2)+1
new_df['Satisfaction_score'] = new_df['Satisfaction_score'].fillna(0)

In [48]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148506 entries, 0 to 148505
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   MSISDN/Number             148506 non-null  float64
 1   Session frequency         148506 non-null  int64  
 2   Dur. (ms)                 148506 non-null  float64
 3   Total_volume (Bytes)      148506 non-null  float64
 4   Engagement_labels         148506 non-null  int32  
 5   Engagement_score          148506 non-null  float64
 6   AVG RTT (ms)              148506 non-null  float64
 7   Avg Bearer TP (kbps)      148506 non-null  float64
 8   TCP Retrans. Vol (Bytes)  148506 non-null  float64
 9   Experience_labels         148506 non-null  int32  
 10  Experience_score          148506 non-null  float64
 11  Satisfaction_score        148506 non-null  float64
dtypes: float64(9), int32(2), int64(1)
memory usage: 12.5 MB


In [49]:
new_df.head(50)

,MSISDN/Number,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels,Engagement_score,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score,Satisfaction_score
0,-0.003363,1,0.05533,-9.844744e-01,2,0.000000,-0.286358,-0.925733,-6.359696e-17,0,0.000000,1.000000
1,-0.003356,1,0.05533,1.938198e+00,0,0.000000,-0.242932,-0.929912,-6.359696e-17,0,0.000000,1.000000
2,-0.003324,1,0.05533,-1.889650e+00,2,0.000000,0.000000,-0.934000,-6.359696e-17,0,0.000000,1.000000
3,-0.003328,1,0.05533,1.798275e+00,0,0.000000,0.000000,-0.924857,-6.359696e-17,0,0.000000,1.000000
4,-0.003348,1,0.05533,2.403649e-01,1,0.000000,0.000000,-0.934000,-6.359696e-17,0,0.000000,1.000000
5,-0.003361,1,0.05533,2.691306e-01,1,0.000000,0.000000,-0.921614,-6.359696e-17,0,0.000000,1.000000
6,-0.003362,1,0.05533,-9.695963e-01,2,0.000000,-0.173072,-0.927070,-6.359696e-17,0,0.000000,1.000000
7,-0.003322,1,0.05533,1.408111e-01,1,0.000000,-0.122019,-0.692225,-1.791441e-01,0,0.000000,1.000000
8,-0.003349,1,0.05533,1.776882e+00,0,0.000000,0.000000,-0.923694,-6.359696e-17,0,0.000000,1.000000
9,-0.003365,1,0.05533,-3.677768e-02,1,0.000000,-0.156358,-0.228460,-1.999040e-01,0,0.000000,1.000000


In [50]:
new_df['Satisfaction_score'].sort_values(ascending=False)[:10]

1817      14.467387
146874    13.354777
37649     12.196516
57811     11.746603
67629     11.630514
90258     11.380157
36204     11.056983
77197     11.035598
64973     10.802204
79053     10.780743
Name: Satisfaction_score, dtype: float64

In [51]:
new_df.sort_values(ascending=False, by='Satisfaction_score')[:10]

,MSISDN/Number,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels,Engagement_score,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score,Satisfaction_score
1817,-0.003363,1,-0.230473,1.474995e+00,0,0.000000,1.630599,13.155008,12.223320,2,26.934775,14.467387
146874,-0.003323,16,-0.230473,-9.436103e-01,2,14.690398,0.203721,9.775711,0.237392,2,10.019157,13.354777
37649,-0.003362,4,-0.230473,-8.082499e-01,2,2.183840,0.329666,7.372795,12.605453,2,20.209191,12.196516
57811,-0.003323,1,-0.230461,-6.511287e-01,1,0.000000,0.253928,7.885140,13.446991,2,21.493207,11.746603
67629,-0.003323,1,-0.230461,2.126456e-01,1,0.000000,0.269730,7.969450,13.115709,2,21.261028,11.630514
90258,-0.003362,1,-0.713202,-1.374139e-16,1,0.000000,-0.151810,21.007296,0.000943,2,20.760314,11.380157
36204,-0.003364,2,-0.230473,-1.224055e+00,2,0.000000,2.545801,13.188554,4.478800,2,20.113965,11.056983
77197,-0.003322,2,-0.206779,6.695080e-01,1,0.000000,0.424629,7.938475,11.807491,2,20.071196,11.035598
64973,-0.003361,4,-0.230461,1.391034e+00,0,0.000000,-0.035724,6.352619,13.389267,2,19.604408,10.802204
79053,-0.003349,1,-0.443875,-1.374139e-16,1,0.000000,0.857583,6.370326,12.435554,2,19.561486,10.780743


## Task 4.3

Build a regression model to predict the satisfaction score from the engagement and experience numerical metrics.

In [52]:
y = new_df['Satisfaction_score']

In [53]:
cols_drop = ['MSISDN/Number','Satisfaction_score','Engagement_labels','Experience_labels','Experience_score','Engagement_score']
X = new_df.drop(cols_drop,axis=1)

In [54]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148506 entries, 0 to 148505
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Session frequency         148506 non-null  int64  
 1   Dur. (ms)                 148506 non-null  float64
 2   Total_volume (Bytes)      148506 non-null  float64
 3   AVG RTT (ms)              148506 non-null  float64
 4   Avg Bearer TP (kbps)      148506 non-null  float64
 5   TCP Retrans. Vol (Bytes)  148506 non-null  float64
dtypes: float64(5), int64(1)
memory usage: 6.8 MB


In [55]:
y.value_counts()

1.000000    132412
2.046624         1
1.890958         1
2.272980         1
1.519368         1
             ...  
1.875804         1
3.960809         1
3.221544         1
2.360387         1
1.469087         1
Name: Satisfaction_score, Length: 16095, dtype: int64

In [56]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, test_size= 0.25)

In [57]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(111379, 6)
(37127, 6)
(111379,)
(37127,)


In [58]:
lin_reg = LinearRegression()


In [59]:
lin_reg.fit(X_train,y_train)
print(lin_reg.intercept_)
print(lin_reg.coef_)

1.1683583751524635
[ 0.03017033  0.11021657 -0.00684285  0.0071424   0.34898573  0.00499308]


Evaluate the model built

In [60]:
model = lin_reg.predict(X_test)

In [61]:
print(f"Root mean squared error {np.sqrt(mean_squared_error(y_test,model))}")
print(f"Mean squared error {mean_squared_error(y_test,model)}")
print(f"Mean Absolute error {mean_absolute_error(y_test,model)}")

Root mean squared error 0.37531103971228874
Mean squared error 0.14085837652991917
Mean Absolute error 0.22366972466250032


In [62]:
score_check = pd.DataFrame()

In [63]:
score_check['y_test'] = y_test

In [64]:
score_check['model'] = model

In [65]:
score_check

,y_test,model
63312,1.000000,0.860135
32540,1.000000,0.920415
32971,1.000000,0.892716
6396,1.000000,1.120522
43350,1.000000,0.907441
...,...,...
136000,1.000000,0.982461
139153,1.000000,0.881651
29881,1.000000,0.962632
59861,2.870834,2.612562


## Task 4.4

Run a Kmeans to seperate engagement and experience score to two clusters

In [66]:
metrics = new_df[['Engagement_score','Experience_score']]

In [67]:
met_cols = ['Engagement_score','Experience_score']

In [68]:
Kmeans_2 = KMeans(
    init= "random",
    n_clusters = 2,
    n_init= 10,
    max_iter= 300,
    random_state= 42
)

In [69]:
Kmeans_2.fit(metrics)

KMeans(init='random', n_clusters=2, random_state=42)

In [70]:
metrics

,Engagement_score,Experience_score
0,0.0,0.000000
1,0.0,0.000000
2,0.0,0.000000
3,0.0,0.000000
4,0.0,0.000000
...,...,...
148501,0.0,0.938173
148502,0.0,0.000000
148503,0.0,0.000000
148504,0.0,0.000000


In [71]:
metrics['cluster'] = Kmeans_2.labels_

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/2972770247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['cluster'] = Kmeans_2.labels_


## Task 4.5

In [72]:
metrics['Satisfaction_score'] = new_df['Satisfaction_score']

C:\Users\CLASSISH\AppData\Local\Temp/ipykernel_5048/3150825679.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['Satisfaction_score'] = new_df['Satisfaction_score']


Aggregate the average satisfaction and experience score per cluster

In [73]:
satisfied_per_cluster = metrics.groupby('cluster').agg({'Satisfaction_score':["median","mean"]})

In [74]:
satisfied_per_cluster

Satisfaction_score          
                    median      mean
cluster                             
0                 1.000000  1.044561
1                 3.323308  3.705349

In [75]:
experience_per_cluster = metrics.groupby('cluster').agg({'Experience_score':["median","mean"]})


In [76]:
experience_per_cluster

Experience_score          
                  median      mean
cluster                           
0                0.00000  0.076590
1                4.62993  5.389693

In [77]:
new_df

,MSISDN/Number,Session frequency,Dur. (ms),Total_volume (Bytes),Engagement_labels,Engagement_score,AVG RTT (ms),Avg Bearer TP (kbps),TCP Retrans. Vol (Bytes),Experience_labels,Experience_score,Satisfaction_score
0,-0.003363,1,0.055330,-0.984474,2,0.0,-0.286358,-0.925733,-6.359696e-17,0,0.000000,1.000000
1,-0.003356,1,0.055330,1.938198,0,0.0,-0.242932,-0.929912,-6.359696e-17,0,0.000000,1.000000
2,-0.003324,1,0.055330,-1.889650,2,0.0,0.000000,-0.934000,-6.359696e-17,0,0.000000,1.000000
3,-0.003328,1,0.055330,1.798275,0,0.0,0.000000,-0.924857,-6.359696e-17,0,0.000000,1.000000
4,-0.003348,1,0.055330,0.240365,1,0.0,0.000000,-0.934000,-6.359696e-17,0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
148501,-0.003361,1,-0.561190,0.140811,1,0.0,0.277626,2.007879,-7.639381e-02,2,0.938173,1.469087
148502,-0.003368,1,-0.294231,1.953256,0,0.0,-0.370625,-0.919989,-6.359696e-17,0,0.000000,1.000000
148503,-0.003363,1,-0.087749,0.692219,1,0.0,-0.353911,-0.923573,-6.359696e-17,0,0.000000,1.000000
148504,-0.003380,2,-0.084308,0.015305,1,0.0,-0.271393,-0.924251,-6.359696e-17,0,0.000000,1.000000


In [78]:
metrics

,Engagement_score,Experience_score,cluster,Satisfaction_score
0,0.0,0.000000,0,1.000000
1,0.0,0.000000,0,1.000000
2,0.0,0.000000,0,1.000000
3,0.0,0.000000,0,1.000000
4,0.0,0.000000,0,1.000000
...,...,...,...,...
148501,0.0,0.938173,0,1.469087
148502,0.0,0.000000,0,1.000000
148503,0.0,0.000000,0,1.000000
148504,0.0,0.000000,0,1.000000


## Task 4.6

Export final table to SQL database

In [79]:
sql_df = pd.DataFrame()

In [80]:
sql_df['MSISDN/Number'] = new_df['MSISDN/Number']

In [81]:
cols_sql_df = ['Engagement_score','Experience_score','Satisfaction_score']
sql_df[cols_sql_df] = metrics[cols_sql_df]

In [82]:
sql_df

,MSISDN/Number,Engagement_score,Experience_score,Satisfaction_score
0,-0.003363,0.0,0.000000,1.000000
1,-0.003356,0.0,0.000000,1.000000
2,-0.003324,0.0,0.000000,1.000000
3,-0.003328,0.0,0.000000,1.000000
4,-0.003348,0.0,0.000000,1.000000
...,...,...,...,...
148501,-0.003361,0.0,0.938173,1.469087
148502,-0.003368,0.0,0.000000,1.000000
148503,-0.003363,0.0,0.000000,1.000000
148504,-0.003380,0.0,0.000000,1.000000


In [83]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://',echo = False)

Need to comment the code below because the sqlit database name has been created already and it does not duplicate, so it will throw an error if reran.

In [94]:
#conn = sql.connect('sql_df.db')
#sql_df.to_sql('sqlite_df', conn)

Code below to read the database in my notebook

In [96]:
conn = sql.connect('sqlite_df.db')
sql_db = pd.read_sql('SELECT * FROM sql_df WHERE Satisfaction_score == 1', conn)

In [97]:
sql_db

,index,MSISDN/Number,Engagement_score,Experience_score,Satisfaction_score
0,0,-0.003363,-0.063274,-0.19422,1.0
1,1,-0.003356,-0.063274,-0.19422,1.0
2,2,-0.003324,-0.063274,-0.19422,1.0
3,3,-0.003328,-0.063274,-0.19422,1.0
4,4,-0.003348,-0.063274,-0.19422,1.0
...,...,...,...,...,...
139454,148501,-0.003361,-0.063274,-0.19422,1.0
139455,148502,-0.003368,-0.063274,-0.19422,1.0
139456,148503,-0.003363,-0.063274,-0.19422,1.0
139457,148504,-0.003380,-0.063274,-0.19422,1.0


## Task 4.7

Create files to export model as, for deployment purpose.

In [98]:
import pickle
with open('model_pkl_1', 'wb') as files:
    pickle.dump(lin_reg, files)

In [99]:
import joblib

In [100]:
joblib.dump(lin_reg, 'model_joblib')

['model_joblib']